由于jupyter notebook的shell实在过于垃圾  
所以shell的代码全部放到.sh里面得了  
使用前记得chmod +x file_name.sh

In [38]:
!bash ./shell_script/1_sra_to_fastq.sh

./shell_script/1_sra_to_fastq.sh: line 5: fasterq-dump: command not found
./shell_script/1_sra_to_fastq.sh: line 5: fasterq-dump: command not found
